#### [그룹화 처리]
- Series/DataFrame에 groupby() 메서드를 사용
- Series/DataFrameGroupBY Object 결과로 반환

In [2]:
# [1]
import pandas as pd
import df_util as util

In [3]:
# [2] 데이터 준비
file = r'C:\Users\a\Desktop\EX_PANDAS\DAY_06\titanic.csv'

In [4]:
# [3] csv => dataFrame
df1 = pd.read_csv(file)

In [5]:
# [4] 데이터 확인
util.checkDataFrame(df1,df1)


[     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  a

In [6]:
# [5] 분석 주제 ==> 성별에 따른 나이 분포
# - 분석 컬럼 즉, 성별과 나이 컬럼
# - loc[행,열]
gender_ageDF = df1.loc[:,['age','sex']] 

In [7]:
gender_ageDF = df1[['age','sex']]

In [8]:
# - [5-1] 데이터 전처리 : 결측치, 중복값
# - 결측치 : isna(),isnull()
gender_ageDF.isna().sum()

age    177
sex      0
dtype: int64

In [9]:
# -- 결측치 처리 ==> 성별에 따라 치환 
# ==> 성별에 따른 그룹화
# gender_ageDF['sex'] = 'male'
groupObj = gender_ageDF.groupby('sex')

In [10]:
# -- Group 객체의 속성
# -- Groups속성 : 그룹화된 그룹에 속하는 인덱스 정보 저장 Dict 형태

print(groupObj.groups.keys())
print(groupObj.groups.values())


dict_keys(['female', 'male'])
dict_values([Index([  1,   2,   3,   8,   9,  10,  11,  14,  15,  18,
       ...
       866, 871, 874, 875, 879, 880, 882, 885, 887, 888],
      dtype='int64', length=314), Index([  0,   4,   5,   6,   7,  12,  13,  16,  17,  20,
       ...
       873, 876, 877, 878, 881, 883, 884, 886, 889, 890],
      dtype='int64', length=577)])


In [11]:
# -- Groups속성 : 각 그룹의 인덱스 속성을 저장 

# print(groupObj.indices)
print(groupObj.indices.keys())
# print(groupObj.())

dict_keys(['female', 'male'])


In [12]:
# -- get_group(key) 메서드 : 그룹화된 그룹들 중에서 특정 그룹 데이터 읽기 메서드
#                         DataFrame 반환
groupObj.get_group('male')

,age,sex
0,22.0,male
4,35.0,male
5,NaN,male
6,54.0,male
7,2.0,male
...,...,...
883,28.0,male
884,25.0,male
886,27.0,male
889,26.0,male


In [13]:
for key in groupObj.groups.keys():
    print(f'\n[{key}]')
    print(groupObj.get_group(key).ndim,groupObj.get_group(key).shape)
    print(groupObj.get_group(key).min(),groupObj.get_group(key).max())


[female]
2 (314, 2)
age      0.75
sex    female
dtype: object age      63.0
sex    female
dtype: object

[male]
2 (577, 2)
age    0.42
sex    male
dtype: object age    80.0
sex    male
dtype: object


In [14]:
# - 그룹별로 집계연산 수행
print(groupObj.count())
print(groupObj.max())
print(groupObj.min())
print(groupObj.mean())
print(groupObj.median())

        age
sex        
female  261
male    453
         age
sex         
female  63.0
male    80.0
         age
sex         
female  0.75
male    0.42
              age
sex              
female  27.915709
male    30.726645
         age
sex         
female  27.0
male    29.0


In [15]:
groupObj.agg(['min','max','mean','median','sum'])

age                                  
         min   max       mean median       sum
sex                                           
female  0.75  63.0  27.915709   27.0   7286.00
male    0.42  80.0  30.726645   29.0  13919.17

In [16]:
resultDF = groupObj.agg(['min','max','mean','median','sum'])

In [17]:
resultDF.columns

MultiIndex([('age',    'min'),
            ('age',    'max'),
            ('age',   'mean'),
            ('age', 'median'),
            ('age',    'sum')],
           )

In [18]:
resultDF[[('age','min'),('age','mean')]]

age           
         min       mean
sex                    
female  0.75  27.915709
male    0.42  30.726645

In [19]:
resultDF[('age','min')]['female']

0.75

In [20]:
# Series에서 Series 연산
resultDF[('age','max')] - resultDF[('age','min')]

sex
female    62.25
male      79.58
dtype: float64

In [24]:
df1.alone

0      False
1      False
2       True
3      False
4       True
       ...  
886     True
887     True
888    False
889     True
890     True
Name: alone, Length: 891, dtype: bool